# List devices
9.2.2022, Sakari Lukkarinen<br>
Metropolia University of Applied Sciences<br>

## Introduction

The aim of this Notebook is to get familiar with the [bleak](https://github.com/hbldh/bleak) - Bluetooth Low Energy Platform for Python. In order to use it, you need first to install it to your Python environment. The easiest way is to remove the comment mark from the `!pip install bleak` code row in the Setup part of this Notebook. 

You also need to install [nest_asyncio](https://github.com/erdewit/nest_asyncio) library in order that `asyncio` works properly in Notebook. There is also commented line ready for that in the Setup part, search for `!pip install nest_asyncio` code.

The first part of this Notebook lists all Bluetooth devices that can be discovered with the [BleakScanner](https://github.com/hbldh/bleak). When the proper Polar heart rate device is found, you need to copy the heart rate device MAC-address to the second part of the Notebook, which shows all the details of the device, including the battery level and measurements types (ECG, PPG, Acceleration, PP interval, etc.) supported by that device.

For more information how the `bleak` library works, see https://bleak.readthedocs.io/en/latest/scanning.html#

## Setup

- If these libraries are missing from your installation, remove the comment and run once.
- When installation is ready, add the comment back.

In [ ]:
# !pip install nest_asyncio
# !pip install bleak

In [ ]:
import nest_asyncio
nest_asyncio.apply()

import asyncio
from bleak import BleakScanner, BleakClient

## List devices

- Lists all Bluetooth devices found
- Shows the MAC-addresses and the device names
- Copy the shown MAC address for Polar device

In [ ]:
devices = []

async def list_devices():
    devices = await BleakScanner.discover()
    for d in devices:
        print(d)
        #if "Polar" in d:
        #    print('Polar device found!')
        print(d.address)
        print()

asyncio.run(list_devices())

## Show details for selected device

- Shows the details of the device
    - Manufacturer
    - Model number
    - Battery level
    - Data stream options
    
**Remember to change the address!**

In [ ]:
## Change the address!!!
ADDRESS = "E3:7B:AA:2D:D5:CD"

MANUFACTURER_NAME_UUID = "00002a29-0000-1000-8000-00805f9b34fb"
MODEL_NBR_UUID = "00002a24-0000-1000-8000-00805f9b34fb"
BATTERY_LEVEL_UUID = "00002a19-0000-1000-8000-00805f9b34fb"
PMD_CONTROL = "FB005C81-02E7-F387-1CAD-8ACD2D8DF0C8"

async def show_details(address):
    client = BleakClient(address)
    try:
        await client.connect()
        manufacturer = await client.read_gatt_char(MANUFACTURER_NAME_UUID)
        print("Manufacturer:  {0}".format("".join(map(chr, manufacturer))))

        model_number = await client.read_gatt_char(MODEL_NBR_UUID)
        print("Model Number:  {0}".format("".join(map(chr, model_number))))

        battery = await client.read_gatt_char(BATTERY_LEVEL_UUID)
        print("Battery level: {0}%".format(int(battery[0])))
        
        att_read = await client.read_gatt_char(PMD_CONTROL)
        if att_read[0] == 0x0F:
            features = att_read[1]
            print('Supported measurement types:')
            print('    ECG          = ', features&0b000001 != 0)
            print('    Acceleration = ', features&0b000100 != 0)
            print('    PPG          = ', features&0b000010 != 0)
            print('    PP interval  = ', features&0b001000 != 0)
            print('    Gyroscope    = ', features&0b010000 != 0)
            print('    Magnetometer = ', features&0b100000 != 0)

    except Exception as e:
        print(e)
    finally:
        await client.disconnect()

asyncio.run(show_details(ADDRESS))

## More info
- [Polar BLE technical documentation](https://github.com/polarofficial/polar-ble-sdk/tree/master/technical_documentation)
- [Polar measurement data specifications for 3rd party](https://github.com/polarofficial/polar-ble-sdk/blob/master/technical_documentation/Polar_Measurement_Data_Specification.pdf)